# 🧪 Chemical Report Generator — Build & Usage Guide

This tool will let you and your lab partner quickly generate tables of chemical properties 
(density, formula, molecular mass, mp/bp, hazards) **plus structure images** directly in 
a Google Doc. Later, the Doc can be exported as a polished PDF for lab reports.

---

## 1️⃣ Setup & Requirements

### Install Python packages
```bash
pip install pubchempy google-api-python-client google-auth-httplib2 google-auth-oauthlib requests
```

### Google Cloud Setup

1. Create a project in [Google Cloud Console](https://console.cloud.google.com/).

2. Enable Google Docs API and Google Drive API.

3. Create OAuth credentials (Desktop App or Service Account).

4. Download `credentials.json` to your project folder.

5. First run → authenticate in browser → saves `token.json` for future runs.

---

## 2️⃣ Fetching Chemical Data

Use **PubChemPy** to query compound info:

- 🧬 Formula
- ⚖️ Molecular Weight  
- 🌡️ Melting / Boiling Point
- 💧 Density
- ☣️ Hazards (when available)

Augment with your own lab-specific dictionary (psychoactive compounds, stability notes, extraction solvents).

---

## 3️⃣ Getting Chemical Structure Images

Two options:

### Fast (Recommended): PubChem API

Download PNG directly:
```
https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{compound}/PNG
```

Saves time and always returns a decent structure.

### Custom: RDKit

Use RDKit to draw molecules from SMILES strings.

More control, but requires installing RDKit locally.

---

## 4️⃣ Inserting Data into Google Docs

Use the Docs API:

1. Create a table with rows for each property (formula, MM, mp/bp, density, hazards).

2. Insert chemical structure image into the last row.

3. You can format headers with bold and add emojis for readability.

**Example table layout:**

| Property | Value |
|----------|-------|
| Name | Valerenic acid |
| Formula 🧬 | C15H22O2 |
| Mol Mass ⚖️ | 234.33 g/mol |
| Density 💧 | 1.06 g/mL |
| MP/BP 🌡️ | 134–137°C / 375°C |
| Hazards ☣️ | Psychoactive, handle carefully |
| Structure 🖼 | (inserted image) |

---

## 5️⃣ Export as PDF

Once data is in the Doc:

- Use Google Drive API to export as PDF programmatically.
- Or your lab partner can simply use "File → Download as PDF" inside Docs.

---

## 6️⃣ Putting It All Together

Build a Python function like:

```python
def generate_lab_report(compounds, doc_id):
    for compound in compounds:
        data = fetch_pubchem_data(compound)
        image = fetch_pubchem_structure(compound)
        insert_table_in_doc(doc_id, data, image)
```

Script can handle multiple compounds in one run.

---

## 7️⃣ Workflow for Your Lab Partner

1. Open Google Docs template for lab reports.

2. Run the script:
   ```bash
   python lab_report.py "psilocybin" "thc" "valerenic acid"
   ```

3. Script pulls:
   - Data from PubChem + your lab dictionary.
   - Structure image (PNG).
   - Inserts a formatted table in the Doc.

4. He writes the rest of the report (methods, results, discussion).

5. Export as PDF for submission.

---

## 8️⃣ Time Estimate

- **Basic version** (text only): 1–2 hours.
- **With structures + Docs integration**: 4–6 hours.
- **Polished, auto-PDF workflow**: 1–2 days of refinement.

---

## 🚀 Extensions (Future Ideas)

- Build a **Streamlit front-end** → type compound → see table → push to Google Docs.
- Create a **Docs Add-On** (Apps Script) so he types compound names directly in the Doc, and they auto-populate.
- Add **NMR/IR spectra links** from PubChem for deeper analysis.
- Save a **personal compound database** to avoid re-fetching.

In [ ]:
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
import pubchempy as pcp
import requests
import base64
import os
from google.oauth2.credentials import Credentials

SCOPES = ['https://www.googleapis.com/auth/documents',
          'https://www.googleapis.com/auth/drive.file']

_CREDS = None  # in-memory cache

# --- Authenticate to Google (cached + token.json) ---
def authenticate_google():
    global _CREDS
    if _CREDS and _CREDS.valid:
        return _CREDS
    creds_path = r"C:\Users\Windows User\NMR-Project\chem_tools\credentials.json"
    token_path = r"C:\Users\Windows User\NMR-Project\chem_tools\token.json"
    creds = None
    if os.path.exists(token_path):
        creds = Credentials.from_authorized_user_file(token_path, SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(creds_path, SCOPES)
            creds = flow.run_local_server(port=0)
        with open(token_path, 'w') as token:
            token.write(creds.to_json())
    _CREDS = creds
    return creds

def create_google_doc(title="Lab Report"):
    creds = authenticate_google()
    service = build('docs', 'v1', credentials=creds)
    doc = service.documents().create(body={'title': title}).execute()
    doc_id = doc['documentId']
    print(f"Created document: {doc['title']} (ID: {doc_id})")
    print(f"Open it here: https://docs.google.com/document/d/{doc_id}/edit")
    return doc_id

# --- Fetch chemical data from PubChem ---
def fetch_pubchem_data(compound_name):
    try:
        compound = pcp.get_compounds(compound_name, 'name')[0]
        return {
            "Name": compound.iupac_name or compound_name,
            "Formula 🧬": compound.molecular_formula,
            "Mol Mass ⚖️": f"{compound.molecular_weight} g/mol",
        }
    except IndexError:
        print(f"Compound '{compound_name}' not found on PubChem.")
        return None

# --- Fetch structure image as base64 ---
def fetch_structure_image(compound_name):
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{compound_name}/PNG"
    response = requests.get(url)
    if response.status_code == 200:
        encoded = base64.b64encode(response.content).decode('utf-8')
        return encoded
    else:
        print(f"Failed to fetch structure for {compound_name}")
        return None

# --- Notebook-friendly table insertion ---
from googleapiclient.discovery import build

def insert_table_in_doc(doc_id, service, data, image_base64=None):
    """
    Insert a table with data into a Google Doc, optionally inserting an image after the table.

    Parameters:
        doc_id (str): Google Doc ID.
        service: Authenticated Docs API service object.
        data (list of list of str): 2D list representing rows and columns.
        image_base64 (str, optional): Base64-encoded PNG image to insert after the table.
    """
    requests_body = []

    # --- Insert table ---
    num_rows = len(data)
    num_cols = len(data[0]) if num_rows > 0 else 0

    requests_body.append({
        "insertTable": {
            "rows": num_rows,
            "columns": num_cols,
            "location": {"index": 1}  # index 1 = right after document start
        }
    })

    # --- Fill table with text ---
    for r_idx, row in enumerate(data):
        for c_idx, cell_text in enumerate(row):
            requests_body.append({
                "insertText": {
                    "location": {
                        "index": 2 + r_idx * num_cols + c_idx  # approximate index inside table
                    },
                    "text": cell_text
                }
            })

    # --- Insert image after the table ---
    if image_base64:
        requests_body.append({
            "insertInlineImage": {
                "location": {"index": num_rows * num_cols + 2},  # right after table
                "uri": f"data:image/png;base64,{image_base64}",
                "objectSize": {
                    "height": {"magnitude": 100, "unit": "PT"},
                    "width": {"magnitude": 100, "unit": "PT"}
                }
            }
        })

    # --- Execute batchUpdate ---
    service.documents().batchUpdate(documentId=doc_id, body={"requests": requests_body}).execute()
    print("Table inserted into Google Doc.")


In [23]:
# --- FINAL WORKING LAB TABLE GENERATOR ---

def create_final_lab_table(compound_name):
    """
    Creates the perfect lab table with proper empty cell handling
    """
    print(f"🧪 Creating final lab table for: {compound_name}")
    
    # Get compound data
    try:
        compound = pcp.get_compounds(compound_name, 'name')[0]
        name = compound.iupac_name or compound_name
        formula = compound.molecular_formula or "N/A"
        
        try:
            mol_weight = f"{compound.molecular_weight:.2f} g/mol"
            mw_value = float(str(compound.molecular_weight))
            if mw_value < 200:
                state = "Liquid/Solid"
            else:
                state = "Solid"
        except:
            mol_weight = "N/A"
            state = "Unknown"
            
    except:
        name = compound_name
        formula = "C?H?O?"
        mol_weight = "N/A"
        state = "Unknown"
    
    # Perfect table layout
    headers = [
        "Structure 🖼️", 
        "Name + Formula", 
        "MP/BP 🌡️", 
        "Mol Mass ⚖️", 
        "Density 💧", 
        "State 🧊", 
        "Hazards ⚠️"
    ]
    
    row1 = [
        "[Structure Image]",
        name,
        "See literature",
        mol_weight,
        "See literature", 
        state,
        "[iPad Entry]"
    ]
    
    row2 = [
        " ",  # Space instead of empty
        formula,
        " ",  # Space instead of empty
        " ",
        " ",
        " ",
        " "
    ]
    
    table_data = [headers, row1, row2]
    
    print("📋 Final lab table layout:")
    for i, row in enumerate(table_data):
        print(f"Row {i}: {row}")
    
    # Insert using manual approach to handle empty cells
    num_rows = len(table_data)
    num_cols = len(table_data[0])
    
    # Get document end
    doc = service.documents().get(documentId=doc_id).execute()
    content = doc.get('body').get('content')
    end_index = content[-1].get('endIndex') - 1
    
    # Create table
    requests_body = [{
        "insertTable": {
            "rows": num_rows,
            "columns": num_cols,
            "location": {"index": end_index}
        }
    }]
    
    service.documents().batchUpdate(documentId=doc_id, body={"requests": requests_body}).execute()
    print("📋 Table structure created")
    
    # Fill cells one by one, skipping truly empty ones
    for row_idx, row_data in enumerate(table_data):
        for col_idx, cell_text in enumerate(row_data):
            if cell_text and cell_text.strip():  # Only insert non-empty text
                # Get fresh document
                doc = service.documents().get(documentId=doc_id).execute()
                content = doc.get('body').get('content')
                
                # Find table
                table_element = None
                for element in content:
                    if 'table' in element:
                        table_element = element['table']
                        break
                
                if table_element:
                    table_row = table_element['tableRows'][row_idx]
                    cell = table_row['tableCells'][col_idx]
                    cell_content = cell['content'][0]
                    start_index = cell_content['startIndex']
                    
                    single_request = [{
                        "insertText": {
                            "location": {"index": start_index},
                            "text": str(cell_text)
                        }
                    }]
                    
                    service.documents().batchUpdate(documentId=doc_id, body={"requests": single_request}).execute()
                    print(f"✅ Inserted '{cell_text}' into [{row_idx},{col_idx}]")
    
    print("🎉 PERFECT LAB TABLE COMPLETED!")
    print(f"📖 View: https://docs.google.com/document/d/{doc_id}/edit")
    print("💡 Structure column: Add image manually")
    print("💡 Hazards column: Fill with iPad")
    
    return True

# Create the perfect lab table!
create_final_lab_table("valerenic acid")

🧪 Creating final lab table for: valerenic acid
📋 Final lab table layout:
Row 0: ['Structure 🖼️', 'Name + Formula', 'MP/BP 🌡️', 'Mol Mass ⚖️', 'Density 💧', 'State 🧊', 'Hazards ⚠️']
Row 1: ['[Structure Image]', '(E)-3-[(4S,7R,7aR)-3,7-dimethyl-2,4,5,6,7,7a-hexahydro-1H-inden-4-yl]-2-methylprop-2-enoic acid', 'See literature', 'N/A', 'See literature', 'Unknown', '[iPad Entry]']
Row 2: [' ', 'C15H22O2', ' ', ' ', ' ', ' ', ' ']
📋 Table structure created
✅ Inserted 'Structure 🖼️' into [0,0]
✅ Inserted 'Name + Formula' into [0,1]
✅ Inserted 'MP/BP 🌡️' into [0,2]
✅ Inserted 'Mol Mass ⚖️' into [0,3]
✅ Inserted 'Density 💧' into [0,4]
✅ Inserted 'State 🧊' into [0,5]
✅ Inserted 'Hazards ⚠️' into [0,6]
✅ Inserted '[Structure Image]' into [1,0]
✅ Inserted '(E)-3-[(4S,7R,7aR)-3,7-dimethyl-2,4,5,6,7,7a-hexahydro-1H-inden-4-yl]-2-methylprop-2-enoic acid' into [1,1]
✅ Inserted 'See literature' into [1,2]
✅ Inserted 'N/A' into [1,3]
✅ Inserted 'See literature' into [1,4]
✅ Inserted 'Unknown' into [1,5]


True

In [24]:
# --- COMPLETE AUTOMATED LAB TABLE GENERATOR ---

def get_chem_info(compound_name):
    """
    Get basic chemical info from PubChem
    """
    try:
        compound = pcp.get_compounds(compound_name, 'name')[0]
        return {
            'Name': compound.iupac_name or compound_name,
            'Formula': compound.molecular_formula,
            'Molar_Mass': f"{compound.molecular_weight:.2f} g/mol",
            'Density': getattr(compound, 'density', 'N/A'),
            'Melting_Point': getattr(compound, 'melting_point', 'N/A'),
            'Boiling_Point': getattr(compound, 'boiling_point', 'N/A'),
        }
    except Exception as e:
        print(f"❌ PubChem error: {e}")
        return None

def analyze_psychoactive_compound(compound_name):
    """
    Specialized analysis for psychoactive compounds with lab-relevant data
    """
    # Enhanced compound database for your lab
    lab_compounds = {
        'psilocybin': {
            'category': 'Tryptamine',
            'precursor': 'Tryptophan',
            'extraction_solvent': 'Methanol/Water',
            'stability': 'Light-sensitive, store dark',
            'bioavailability': 'Low (prodrug)',
            'active_metabolite': 'Psilocin',
            'hazards': 'Schedule I, psychoactive'
        },
        'psilocin': {
            'category': 'Tryptamine', 
            'precursor': 'Psilocybin (dephosphorylation)',
            'extraction_solvent': 'Ethanol',
            'stability': 'Very unstable, oxidizes rapidly',
            'bioavailability': 'High',
            'active_metabolite': 'Self',
            'hazards': 'Schedule I, highly psychoactive'
        },
        'thc': {
            'category': 'Cannabinoid',
            'precursor': 'THCA (decarboxylation)',
            'extraction_solvent': 'Ethanol/CO2',
            'stability': 'Stable, light-sensitive',
            'bioavailability': 'Variable (route dependent)',
            'active_metabolite': '11-OH-THC',
            'hazards': 'Controlled substance, psychoactive'
        },
        'cbd': {
            'category': 'Cannabinoid',
            'precursor': 'CBDA (decarboxylation)', 
            'extraction_solvent': 'Ethanol/CO2',
            'stability': 'Very stable',
            'bioavailability': 'Low oral, higher sublingual',
            'active_metabolite': '7-OH-CBD',
            'hazards': 'Generally safe, check local laws'
        },
        'cbn': {
            'category': 'Cannabinoid',
            'precursor': 'THC (oxidation/aging)',
            'extraction_solvent': 'Ethanol',
            'stability': 'Stable',
            'bioavailability': 'Moderate',
            'active_metabolite': '11-OH-CBN',
            'hazards': 'Mildly psychoactive'
        },
        'valerenic acid': {
            'category': 'Sesquiterpene',
            'precursor': 'Valerian root',
            'extraction_solvent': 'Ethanol/Water',
            'stability': 'Moderately stable',
            'bioavailability': 'Moderate',
            'active_metabolite': 'Various metabolites',
            'hazards': 'Generally safe, sedative'
        }
    }
    
    # Get basic chemical info
    chem_info = get_chem_info(compound_name)
    
    if not chem_info:
        print(f"❌ Could not find chemical data for {compound_name}")
        return None
    
    # Add lab-specific information
    lab_info = lab_compounds.get(compound_name.lower(), {})
    
    if lab_info:
        print(f"\n🧪 Complete Analysis for {compound_name.upper()}:")
        print("="*50)
        print(f"Category: {lab_info['category']}")
        print(f"Precursor: {lab_info['precursor']}")
        print(f"Best Extraction Solvent: {lab_info['extraction_solvent']}")
        print(f"Stability Notes: {lab_info['stability']}")
        print(f"Bioavailability: {lab_info['bioavailability']}")
        print(f"Active Metabolite: {lab_info['active_metabolite']}")
        print(f"Hazards: {lab_info['hazards']}")
    
    return {**chem_info, **lab_info}

def create_automated_lab_table(compound_name):
    """
    Fully automated lab table creation with both PubChem and lab-specific data
    """
    print(f"🚀 Creating AUTOMATED lab table for: {compound_name}")
    
    # Get comprehensive compound analysis
    compound_data = analyze_psychoactive_compound(compound_name)
    
    if not compound_data:
        print(f"❌ Could not analyze {compound_name}")
        return False
    
    # Extract data for table
    name = compound_data.get('Name', compound_name)
    formula = compound_data.get('Formula', 'N/A')
    mol_weight = compound_data.get('Molar_Mass', 'N/A')
    
    # MP/BP handling
    mp = compound_data.get('Melting_Point', 'N/A')
    bp = compound_data.get('Boiling_Point', 'N/A')
    if mp != 'N/A':
        mp_bp = f"MP: {mp}°C"
    elif bp != 'N/A':
        mp_bp = f"BP: {bp}°C"
    else:
        mp_bp = "See literature"
    
    # Density
    density = compound_data.get('Density', 'See literature')
    if density == 'N/A':
        density = "See literature"
    
    # State of matter (intelligent guess)
    try:
        mw_value = float(mol_weight.split()[0])
        if mw_value < 50:
            state = "Gas"
        elif mw_value < 200:
            state = "Liquid/Solid"
        else:
            state = "Solid"
    except:
        state = "Unknown"
    
    # Hazards from lab database
    hazards = compound_data.get('hazards', '[Fill on iPad]')
    
    # Perfect automated table layout
    headers = [
        "Structure 🖼️", 
        "Name + Formula", 
        "MP/BP 🌡️", 
        "Mol Mass ⚖️", 
        "Density 💧", 
        "State 🧊", 
        "Hazards ⚠️"
    ]
    
    row1 = [
        "[Chemical Structure]",
        name,
        mp_bp,
        mol_weight,
        density,
        state,
        hazards
    ]
    
    row2 = [
        " ",  # Empty structure cell
        formula,  # Formula under name
        " ",      # Empty
        " ",      # Empty
        " ",      # Empty
        " ",      # Empty
        " "       # Empty
    ]
    
    table_data = [headers, row1, row2]
    
    print("📋 Automated table layout:")
    for i, row in enumerate(table_data):
        print(f"Row {i}: {row}")
    
    # Insert table using existing infrastructure
    num_rows = len(table_data)
    num_cols = len(table_data[0])
    
    # Get document end
    doc = service.documents().get(documentId=doc_id).execute()
    content = doc.get('body').get('content')
    end_index = content[-1].get('endIndex') - 1
    
    # Create table
    requests_body = [{
        "insertTable": {
            "rows": num_rows,
            "columns": num_cols,
            "location": {"index": end_index}
        }
    }]
    
    service.documents().batchUpdate(documentId=doc_id, body={"requests": requests_body}).execute()
    print("📋 Table structure created")
    
    # Fill cells
    for row_idx, row_data in enumerate(table_data):
        for col_idx, cell_text in enumerate(row_data):
            if cell_text and cell_text.strip():
                # Get fresh document
                doc = service.documents().get(documentId=doc_id).execute()
                content = doc.get('body').get('content')
                
                # Find table
                table_element = None
                for element in content:
                    if 'table' in element:
                        table_element = element['table']
                        break
                
                if table_element:
                    table_row = table_element['tableRows'][row_idx]
                    cell = table_row['tableCells'][col_idx]
                    cell_content = cell['content'][0]
                    start_index = cell_content['startIndex']
                    
                    single_request = [{
                        "insertText": {
                            "location": {"index": start_index},
                            "text": str(cell_text)
                        }
                    }]
                    
                    service.documents().batchUpdate(documentId=doc_id, body={"requests": single_request}).execute()
                    print(f"✅ Inserted '{cell_text}' into [{row_idx},{col_idx}]")
    
    print("🎉 AUTOMATED LAB TABLE COMPLETED!")
    print(f"📖 View: https://docs.google.com/document/d/{doc_id}/edit")
    print("💡 All data automatically populated from PubChem + lab database!")
    
    return True

# Test the fully automated system
print("🧪 Testing AUTOMATED lab table system...")
create_automated_lab_table("psilocybin")

🧪 Testing AUTOMATED lab table system...
🚀 Creating AUTOMATED lab table for: psilocybin
❌ PubChem error: Unknown format code 'f' for object of type 'str'
❌ Could not find chemical data for psilocybin
❌ Could not analyze psilocybin


False

In [25]:
# --- FIXED AUTOMATED LAB TABLE GENERATOR ---

def get_chem_info_fixed(compound_name):
    """
    Get basic chemical info from PubChem with proper error handling
    """
    try:
        compound = pcp.get_compounds(compound_name, 'name')[0]
        
        # Safe molecular weight handling
        try:
            mol_weight = f"{float(compound.molecular_weight):.2f} g/mol"
        except:
            mol_weight = "N/A"
        
        return {
            'Name': compound.iupac_name or compound_name,
            'Formula': compound.molecular_formula or "N/A",
            'Molar_Mass': mol_weight,
            'Density': getattr(compound, 'density', 'N/A'),
            'Melting_Point': getattr(compound, 'melting_point', 'N/A'),
            'Boiling_Point': getattr(compound, 'boiling_point', 'N/A'),
        }
    except Exception as e:
        print(f"❌ PubChem error: {e}")
        return None

def analyze_psychoactive_compound_fixed(compound_name):
    """
    Fixed version with better error handling
    """
    # Enhanced compound database for your lab
    lab_compounds = {
        'psilocybin': {
            'category': 'Tryptamine',
            'precursor': 'Tryptophan',
            'extraction_solvent': 'Methanol/Water',
            'stability': 'Light-sensitive, store dark',
            'bioavailability': 'Low (prodrug)',
            'active_metabolite': 'Psilocin',
            'hazards': 'Schedule I, psychoactive'
        },
        'psilocin': {
            'category': 'Tryptamine', 
            'precursor': 'Psilocybin (dephosphorylation)',
            'extraction_solvent': 'Ethanol',
            'stability': 'Very unstable, oxidizes rapidly',
            'bioavailability': 'High',
            'active_metabolite': 'Self',
            'hazards': 'Schedule I, highly psychoactive'
        },
        'thc': {
            'category': 'Cannabinoid',
            'precursor': 'THCA (decarboxylation)',
            'extraction_solvent': 'Ethanol/CO2',
            'stability': 'Stable, light-sensitive',
            'bioavailability': 'Variable (route dependent)',
            'active_metabolite': '11-OH-THC',
            'hazards': 'Controlled substance, psychoactive'
        },
        'cbd': {
            'category': 'Cannabinoid',
            'precursor': 'CBDA (decarboxylation)', 
            'extraction_solvent': 'Ethanol/CO2',
            'stability': 'Very stable',
            'bioavailability': 'Low oral, higher sublingual',
            'active_metabolite': '7-OH-CBD',
            'hazards': 'Generally safe, check local laws'
        },
        'valerenic acid': {
            'category': 'Sesquiterpene',
            'precursor': 'Valerian root',
            'extraction_solvent': 'Ethanol/Water',
            'stability': 'Moderately stable',
            'bioavailability': 'Moderate',
            'active_metabolite': 'Various metabolites',
            'hazards': 'Generally safe, sedative'
        }
    }
    
    # Get basic chemical info
    chem_info = get_chem_info_fixed(compound_name)
    
    if not chem_info:
        print(f"⚠️ Using fallback data for {compound_name}")
        # Create fallback data
        chem_info = {
            'Name': compound_name,
            'Formula': 'See literature', 
            'Molar_Mass': 'See literature',
            'Density': 'See literature',
            'Melting_Point': 'See literature',
            'Boiling_Point': 'See literature'
        }
    
    # Add lab-specific information
    lab_info = lab_compounds.get(compound_name.lower(), {})
    
    if lab_info:
        print(f"\n🧪 Complete Analysis for {compound_name.upper()}:")
        print("="*50)
        print(f"Category: {lab_info['category']}")
        print(f"Precursor: {lab_info['precursor']}")
        print(f"Best Extraction Solvent: {lab_info['extraction_solvent']}")
        print(f"Stability Notes: {lab_info['stability']}")
        print(f"Bioavailability: {lab_info['bioavailability']}")
        print(f"Active Metabolite: {lab_info['active_metabolite']}")
        print(f"Hazards: {lab_info['hazards']}")
    else:
        print(f"💡 {compound_name} not in specialized database - using PubChem data only")
    
    return {**chem_info, **lab_info}

def create_ultimate_lab_table(compound_name):
    """
    The ultimate automated lab table - handles everything!
    """
    print(f"🚀 Creating ULTIMATE automated lab table for: {compound_name}")
    
    # Get comprehensive compound analysis
    compound_data = analyze_psychoactive_compound_fixed(compound_name)
    
    # Extract data for table
    name = compound_data.get('Name', compound_name)
    formula = compound_data.get('Formula', 'N/A')
    mol_weight = compound_data.get('Molar_Mass', 'N/A')
    
    # MP/BP handling
    mp = compound_data.get('Melting_Point', 'N/A')
    bp = compound_data.get('Boiling_Point', 'N/A')
    if mp != 'N/A' and str(mp) != 'N/A':
        mp_bp = f"MP: {mp}°C"
    elif bp != 'N/A' and str(bp) != 'N/A':
        mp_bp = f"BP: {bp}°C"
    else:
        mp_bp = "See literature"
    
    # Density
    density = compound_data.get('Density', 'See literature')
    if density == 'N/A' or str(density) == 'N/A':
        density = "See literature"
    
    # State of matter
    try:
        if mol_weight != 'N/A' and 'g/mol' in str(mol_weight):
            mw_value = float(str(mol_weight).split()[0])
            if mw_value < 50:
                state = "Gas"
            elif mw_value < 200:
                state = "Liquid/Solid"
            else:
                state = "Solid"
        else:
            state = "See literature"
    except:
        state = "See literature"
    
    # Hazards from lab database or default
    hazards = compound_data.get('hazards', '[Fill on iPad]')
    
    # Ultimate table layout
    headers = [
        "Structure 🖼️", 
        "Name + Formula", 
        "MP/BP 🌡️", 
        "Mol Mass ⚖️", 
        "Density 💧", 
        "State 🧊", 
        "Hazards ⚠️"
    ]
    
    row1 = [
        "[Chemical Structure]",
        name,
        mp_bp,
        mol_weight,
        density,
        state,
        hazards
    ]
    
    row2 = [
        " ",
        formula,
        " ",
        " ",
        " ",
        " ",
        " "
    ]
    
    table_data = [headers, row1, row2]
    
    print("📋 Ultimate automated table:")
    for i, row in enumerate(table_data):
        print(f"Row {i}: {row}")
    
    # Insert table
    num_rows = len(table_data)
    num_cols = len(table_data[0])
    
    # Get document end
    doc = service.documents().get(documentId=doc_id).execute()
    content = doc.get('body').get('content')
    end_index = content[-1].get('endIndex') - 1
    
    # Create table
    requests_body = [{
        "insertTable": {
            "rows": num_rows,
            "columns": num_cols,
            "location": {"index": end_index}
        }
    }]
    
    service.documents().batchUpdate(documentId=doc_id, body={"requests": requests_body}).execute()
    print("📋 Table created")
    
    # Fill cells
    for row_idx, row_data in enumerate(table_data):
        for col_idx, cell_text in enumerate(row_data):
            if cell_text and cell_text.strip():
                doc = service.documents().get(documentId=doc_id).execute()
                content = doc.get('body').get('content')
                
                table_element = None
                for element in content:
                    if 'table' in element:
                        table_element = element['table']
                        break
                
                if table_element:
                    table_row = table_element['tableRows'][row_idx]
                    cell = table_row['tableCells'][col_idx]
                    cell_content = cell['content'][0]
                    start_index = cell_content['startIndex']
                    
                    single_request = [{
                        "insertText": {
                            "location": {"index": start_index},
                            "text": str(cell_text)
                        }
                    }]
                    
                    service.documents().batchUpdate(documentId=doc_id, body={"requests": single_request}).execute()
                    print(f"✅ '{cell_text}' → [{row_idx},{col_idx}]")
    
    print("🎉 ULTIMATE LAB TABLE COMPLETED!")
    print(f"📖 View: https://docs.google.com/document/d/{doc_id}/edit")
    print("💡 Fully automated with PubChem + lab database!")
    
    return True

# Test the ultimate system
print("🧪 Testing ULTIMATE automated system...")
create_ultimate_lab_table("psilocybin")

🧪 Testing ULTIMATE automated system...
🚀 Creating ULTIMATE automated lab table for: psilocybin

🧪 Complete Analysis for PSILOCYBIN:
Category: Tryptamine
Precursor: Tryptophan
Best Extraction Solvent: Methanol/Water
Stability Notes: Light-sensitive, store dark
Bioavailability: Low (prodrug)
Active Metabolite: Psilocin
Hazards: Schedule I, psychoactive
📋 Ultimate automated table:
Row 0: ['Structure 🖼️', 'Name + Formula', 'MP/BP 🌡️', 'Mol Mass ⚖️', 'Density 💧', 'State 🧊', 'Hazards ⚠️']
Row 1: ['[Chemical Structure]', '[3-[2-(dimethylamino)ethyl]-1H-indol-4-yl] dihydrogen phosphate', 'See literature', '284.25 g/mol', 'See literature', 'Solid', 'Schedule I, psychoactive']
Row 2: [' ', 'C12H17N2O4P', ' ', ' ', ' ', ' ', ' ']
📋 Table created
✅ 'Structure 🖼️' → [0,0]
✅ 'Name + Formula' → [0,1]
✅ 'MP/BP 🌡️' → [0,2]
✅ 'Mol Mass ⚖️' → [0,3]
✅ 'Density 💧' → [0,4]
✅ 'State 🧊' → [0,5]
✅ 'Hazards ⚠️' → [0,6]
✅ '[Chemical Structure]' → [1,0]
✅ '[3-[2-(dimethylamino)ethyl]-1H-indol-4-yl] dihydrogen pho

True

In [26]:
# --- BATCH PROCESSING FOR MULTIPLE COMPOUNDS ---

def create_compound_tables(compound_list):
    """
    Create tables for multiple compounds automatically
    Just pass a list of compound names!
    """
    print(f"🧪 Creating tables for {len(compound_list)} compounds...")
    print("="*60)
    
    results = []
    
    for i, compound in enumerate(compound_list, 1):
        print(f"\n📋 Processing {i}/{len(compound_list)}: {compound}")
        print("-" * 40)
        
        try:
            success = create_ultimate_lab_table(compound)
            results.append((compound, success))
            
            if success:
                print(f"✅ {compound} table completed!")
            else:
                print(f"❌ {compound} table failed")
                
        except Exception as e:
            print(f"❌ Error with {compound}: {e}")
            results.append((compound, False))
        
        print("-" * 40)
    
    # Summary
    print(f"\n🎉 BATCH PROCESSING COMPLETE!")
    print("="*60)
    print(f"📊 Results Summary:")
    
    successful = 0
    for compound, success in results:
        status = "✅ SUCCESS" if success else "❌ FAILED"
        print(f"  {compound}: {status}")
        if success:
            successful += 1
    
    print(f"\n📈 {successful}/{len(compound_list)} tables created successfully")
    print(f"📖 View all tables: https://docs.google.com/document/d/{doc_id}/edit")
    
    return results

# --- EASY-TO-USE FUNCTIONS FOR YOUR LAB ---

def quick_compound_table(compound_name):
    """
    Quick single compound table - just call this!
    """
    return create_ultimate_lab_table(compound_name)

def psychoactive_batch():
    """
    Pre-defined batch for common psychoactive compounds
    """
    compounds = ['psilocybin', 'psilocin', 'thc', 'cbd', 'cbn']
    return create_compound_tables(compounds)

def custom_batch(compound_names):
    """
    Custom batch - pass your own list
    """
    return create_compound_tables(compound_names)

# Test with a few compounds
print("🧪 Testing batch processing...")

# Test compounds from your lab database
test_compounds = ['thc', 'cbd', 'valerenic acid']
batch_results = create_compound_tables(test_compounds)

🧪 Testing batch processing...
🧪 Creating tables for 3 compounds...

📋 Processing 1/3: thc
----------------------------------------
🚀 Creating ULTIMATE automated lab table for: thc

🧪 Complete Analysis for THC:
Category: Cannabinoid
Precursor: THCA (decarboxylation)
Best Extraction Solvent: Ethanol/CO2
Stability Notes: Stable, light-sensitive
Bioavailability: Variable (route dependent)
Active Metabolite: 11-OH-THC
Hazards: Controlled substance, psychoactive
📋 Ultimate automated table:
Row 0: ['Structure 🖼️', 'Name + Formula', 'MP/BP 🌡️', 'Mol Mass ⚖️', 'Density 💧', 'State 🧊', 'Hazards ⚠️']
Row 1: ['[Chemical Structure]', '(6aR,10aR)-6,6,9-trimethyl-3-pentyl-6a,7,8,10a-tetrahydrobenzo[c]chromen-1-ol', 'See literature', '314.50 g/mol', 'See literature', 'Solid', 'Controlled substance, psychoactive']
Row 2: [' ', 'C21H30O2', ' ', ' ', ' ', ' ', ' ']
📋 Table created
✅ 'Structure 🖼️' → [0,0]
✅ 'Name + Formula' → [0,1]
✅ 'MP/BP 🌡️' → [0,2]
✅ 'Mol Mass ⚖️' → [0,3]
✅ 'Density 💧' → [0,4]
✅ 'Stat

In [35]:
# --- UTILITY FUNCTIONS ---

def create_fresh_doc(title="Clean Lab Report"):
    """
    Create a fresh Google Doc for clean tables
    """
    fresh_doc_id = create_google_doc(title)
    print(f"📄 Fresh document created: {title}")
    print(f"📖 Open: https://docs.google.com/document/d/{fresh_doc_id}/edit")
    return fresh_doc_id

def clear_and_create_tables(compound_list, doc_title="Lab Report - Clean"):
    """
    Start fresh and create clean tables
    """
    # Create fresh doc
    new_doc_id = create_fresh_doc(doc_title)
    # Reuse a single auth session
    new_service, _ = get_services()  # docs, drive

    # Update global variables for new doc
    global doc_id, service
    doc_id = new_doc_id
    service = new_service

    # Create clean tables
    print(f"\n🧪 Creating clean tables in fresh document...")
    create_clean_batch(compound_list)

    return new_doc_id

# --- SIMPLE ONE-LINER FUNCTIONS ---

def single_compound(compound_name):
    """
    One-liner: Create single compound table
    """
    return create_spaced_lab_table(compound_name)

def psychoactive_compounds():
    """
    One-liner: Create tables for all psychoactive compounds
    """
    compounds = ['psilocybin', 'psilocin', 'thc', 'cbd', 'cbn']
    return create_clean_batch(compounds)

def fresh_psychoactive_report():
    """
    One-liner: Fresh doc with all psychoactive compounds
    """
    compounds = ['psilocybin', 'psilocin', 'thc', 'cbd', 'cbn']
    return clear_and_create_tables(compounds, "Psychoactive Compounds Report")

# Quick test of single compound
print("🧪 Testing single compound function...")
single_compound("valerenic acid")

🧪 Testing single compound function...
🧪 Creating spaced lab table for: valerenic acid

🧪 Complete Analysis for VALERENIC ACID:
Category: Sesquiterpene
Precursor: Valerian root
Best Extraction Solvent: Ethanol/Water
Stability Notes: Moderately stable
Bioavailability: Moderate
Active Metabolite: Various metabolites
Hazards: Generally safe, sedative
📝 Added title for valerenic acid
📋 Created table structure
✅ 'Structure 🖼️...' → Cell[0,0]
✅ 'Name + Formula...' → Cell[0,1]
✅ 'MP/BP 🌡️...' → Cell[0,2]
✅ 'Mol Mass ⚖️...' → Cell[0,3]
✅ 'Density 💧...' → Cell[0,4]
✅ 'State 🧊...' → Cell[0,5]
✅ 'Hazards ⚠️...' → Cell[0,6]
✅ '[Chemical Structure]...' → Cell[1,0]
✅ '(E)-3-[(4S,7R,7aR)-3,7-dimethy...' → Cell[1,1]
✅ 'See literature...' → Cell[1,2]
✅ '234.33 g/mol...' → Cell[1,3]
✅ 'See literature...' → Cell[1,4]
✅ 'Solid...' → Cell[1,5]
✅ 'Generally safe, sedative...' → Cell[1,6]
✅ 'C15H22O2...' → Cell[2,1]
🎉 valerenic acid table completed with proper spacing!


True

In [38]:
# RDKit-powered lab table → Google Doc (image + data inserted automatically)

from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseUpload
from google.oauth2.credentials import Credentials
import pubchempy as pcp
import io
import time

# RDKit + PIL
try:
    from rdkit import Chem
    from rdkit.Chem import Draw
except Exception as e:
    raise RuntimeError("RDKit is required. Install with: pip install rdkit-pypi") from e

try:
    from PIL import Image
except Exception as e:
    raise RuntimeError("Pillow is required. Install with: pip install pillow") from e

# Reuse your existing authenticate_google() from earlier cells
# SCOPES already include Docs + Drive.file in your notebook

def get_services():
    """Build Docs + Drive services using your cached auth."""
    creds = authenticate_google()
    docs = build('docs', 'v1', credentials=creds)
    drive = build('drive', 'v3', credentials=creds)
    return docs, drive

def ensure_doc(docs_service, doc_id=None, title="Lab Report"):
    """Use existing doc_id or create a new Doc and return its id."""
    if doc_id:
        return doc_id
    doc = docs_service.documents().create(body={"title": title}).execute()
    return doc["documentId"]

def draw_structure_png(smiles, size=(300, 300)):
    """Return PNG bytes for an RDKit-rendered molecule."""
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    img = Draw.MolToImage(mol, size=size)
    buf = io.BytesIO()
    img.save(buf, format="PNG")
    return buf.getvalue()

def upload_png_to_drive_public(drive_service, png_bytes, filename):
    """Upload PNG to Drive, make it public, and return a viewable URL."""
    media = MediaIoBaseUpload(io.BytesIO(png_bytes), mimetype='image/png', resumable=False)
    file = drive_service.files().create(
        body={'name': filename, 'mimeType': 'image/png'},
        media_body=media,
        fields='id'
    ).execute()
    file_id = file['id']
    # Make public
    drive_service.permissions().create(
        fileId=file_id,
        body={'type': 'anyone', 'role': 'reader'}
    ).execute()
    # Public view URL that works for Docs API
    return f"https://drive.google.com/uc?export=view&id={file_id}"

def get_pubchem_core(compound_name):
    """Fetch core data + SMILES from PubChem (safe fallbacks)."""
    name = compound_name
    formula = "See literature"
    mol_mass = "See literature"
    smiles = None

    try:
        c = pcp.get_compounds(compound_name, 'name')[0]
        name = c.iupac_name or compound_name
        if getattr(c, "molecular_formula", None):
            formula = c.molecular_formula
        try:
            if getattr(c, "molecular_weight", None) is not None:
                mol_mass = f"{float(c.molecular_weight):.2f} g/mol"
        except:
            mol_mass = "See literature"
        smiles = getattr(c, "canonical_smiles", None)
    except Exception as e:
        print(f"⚠️ PubChem lookup issue for '{compound_name}': {e}")

    # Heuristics
    mp_bp = "See literature"
    density = "See literature"
    try:
        mw_val = float(mol_mass.split()[0]) if "g/mol" in mol_mass else None
        if mw_val is None:
            state = "See literature"
        elif mw_val < 50:
            state = "Gas"
        elif mw_val < 200:
            state = "Liquid/Solid"
        else:
            state = "Solid"
    except:
        state = "See literature"

    hazards = "[Fill in]"

    return {
        "name": name,
        "formula": formula,
        "mol_mass": mol_mass,
        "mp_bp": mp_bp,
        "density": density,
        "state": state,
        "hazards": hazards,
        "smiles": smiles
    }

def _get_last_table(docs_service, doc_id):
    """Return the last table object from the Doc body."""
    doc = docs_service.documents().get(documentId=doc_id).execute()
    content = doc.get('body', {}).get('content', [])
    for element in reversed(content):
        if 'table' in element:
            return element['table']
    return None

def _cell_start_index(docs_service, doc_id, table_row_idx, table_col_idx):
    """Fetch current startIndex for a given cell (re-reads the doc each time)."""
    table = _get_last_table(docs_service, doc_id)
    if not table:
        return None
    row = table['tableRows'][table_row_idx]
    cell = row['tableCells'][table_col_idx]
    # Each cell has at least one StructuralElement (a Paragraph) with startIndex
    return cell['content'][0]['startIndex']

def insert_rdkit_lab_table(compound_name, doc_id=None, doc_title="Lab Report"):
    """
    Create/append a 3x7 table:
    [Structure | Name+Formula | MP/BP | Mol Mass | Density | State | Hazards]
    Inserts RDKit image in the Structure cell, and fills the rest from PubChem.
    Returns the Google Doc ID.
    """
    docs, drive = get_services()
    doc_id = ensure_doc(docs, doc_id, doc_title)

    # Get data
    data = get_pubchem_core(compound_name)

    # RDKit structure as a public Drive image
    struct_url = None
    if data["smiles"]:
        png = draw_structure_png(data["smiles"], size=(420, 420))
        if png:
            struct_url = upload_png_to_drive_public(drive, png, f"{compound_name}_structure.png")

    # Table content
    headers = ["Structure 🖼️", "Name + Formula", "MP/BP 🌡️", "Mol Mass ⚖️", "Density 💧", "State 🧊", "Hazards ⚠️"]
    row1 = [
        "",  # image goes here
        data["name"],
        data["mp_bp"],
        data["mol_mass"],
        data["density"],
        data["state"],
        data["hazards"]
    ]
    row2 = ["", data["formula"], "", "", "", "", ""]
    table_data = [headers, row1, row2]

    # Insert empty table at end of document
    doc = docs.documents().get(documentId=doc_id).execute()
    end_index = doc['body']['content'][-1]['endIndex'] - 1
    docs.documents().batchUpdate(
        documentId=doc_id,
        body={
            "requests": [
                {
                    "insertTable": {
                        "rows": len(table_data),
                        "columns": len(table_data[0]),
                        "location": {"index": end_index}
                    }
                }
            ]
        }
    ).execute()

    # Fill cells (insert text one-by-one, re-reading indices to avoid offset issues)
    for r_idx, row in enumerate(table_data):
        for c_idx, text in enumerate(row):
            if r_idx == 1 and c_idx == 0:
                # Structure image cell
                if struct_url:
                    start_idx = _cell_start_index(docs, doc_id, r_idx, c_idx)
                    if start_idx is not None:
                        docs.documents().batchUpdate(
                            documentId=doc_id,
                            body={
                                "requests": [
                                    {
                                        "insertInlineImage": {
                                            "location": {"index": start_idx},
                                            "uri": struct_url,
                                            "objectSize": {
                                                "height": {"magnitude": 140, "unit": "PT"},
                                                "width": {"magnitude": 140, "unit": "PT"}
                                            }
                                        }
                                    }
                                ]
                            }
                        ).execute()
                continue

            if text and str(text).strip():
                start_idx = _cell_start_index(docs, doc_id, r_idx, c_idx)
                if start_idx is not None:
                    docs.documents().batchUpdate(
                        documentId=doc_id,
                        body={
                            "requests": [
                                {
                                    "insertText": {
                                        "location": {"index": start_idx},
                                        "text": str(text)
                                    }
                                }
                            ]
                        }
                    ).execute()

    # Optional: bold header row
    try:
        for c_idx in range(len(headers)):
            start_idx = _cell_start_index(docs, doc_id, 0, c_idx)
            if start_idx is None:
                continue
            # Re-fetch end index of header cell
            table = _get_last_table(docs, doc_id)
            cell = table['tableRows'][0]['tableCells'][c_idx]
            end_idx = cell['content'][0]['endIndex']
            docs.documents().batchUpdate(
                documentId=doc_id,
                body={
                    "requests": [
                        {
                            "updateTextStyle": {
                                "range": {"startIndex": start_idx, "endIndex": end_idx},
                                "textStyle": {"bold": True},
                                "fields": "bold"
                            }
                        }
                    ]
                }
            ).execute()
    except Exception:
        pass

    print(f"✅ Inserted RDKit table for '{compound_name}'")
    print(f"🔗 https://docs.google.com/document/d/{doc_id}/edit")
    return doc_id


In [39]:
# Simple runner: insert RDKit table and print Doc link
compound = "valerenic acid"  # change to any compound name
existing_doc_id = "1umx0PYA9Y9GWImD3dnSwA5NP--sqCN3DSznm7qXyGcc"       # paste an existing Doc ID to append, or leave as None to create a new Doc

doc_id = insert_rdkit_lab_table(compound, doc_id=existing_doc_id, doc_title="Lab Report")

url = f"https://docs.google.com/document/d/{doc_id}/edit"
print("🔗 Google Doc link:", url)

# Return the URL so it's clickable in notebook output
url

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=78955014152-ol5tfcbucd9tmvkjmhfs3bi74rkq1fel.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A51525%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocuments+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.file&state=uLgBK3aevZtt7OyijT2MJOrIDHhd4M&access_type=offline
✅ Inserted RDKit table for 'valerenic acid'
🔗 https://docs.google.com/document/d/1umx0PYA9Y9GWImD3dnSwA5NP--sqCN3DSznm7qXyGcc/edit
🔗 Google Doc link: https://docs.google.com/document/d/1umx0PYA9Y9GWImD3dnSwA5NP--sqCN3DSznm7qXyGcc/edit


'https://docs.google.com/document/d/1umx0PYA9Y9GWImD3dnSwA5NP--sqCN3DSznm7qXyGcc/edit'